# AI-Powered Academic Research Workshop
## Using OpenAI API for Literature Reviews and Paper Analysis

This notebook provides practical examples of using AI tools for academic research workflows.

### Learning goals
- Understand what an API is and how to call it from Python
- Learn to design and customize prompt templates for repeatable tasks
- Use simple `for` loops to automate common research workflows
- Structure outputs (JSON) so they are easy to store, filter, and analyze

### What you’ll build
- A single-paper analyzer that extracts key fields into JSON
- A batch processor that loops over many papers with a progress bar
- A relevance rater that scores papers against your research question
- A short literature synthesis generated from multiple items

### How to use this notebook
- Run cells top-to-bottom the first time
- Edit prompts in the `Prompt templates` section; re-run only the cells that depend on them
- Use the `Loop patterns` section to demonstrate how iteration works step by step

## New to APIs? Quick primer

- **What is an API?** A way for your code to "ask" another service (like an AI model) to do something and return a result — like ordering from a menu and the kitchen delivers the dish.
- **Why use it here?** We send a text prompt to an AI model and get a text response back.
- **What do you need?**
  - An account/key to prove who you are 
  - A small bit of setup to let Python talk to the service
- **Costs/tokens:** Models bill per token (roughly word pieces). We'll show how to estimate and keep requests efficient.

### How a request works (mental model)
1) You prepare a prompt (your instructions)
2) Your code sends it to the API endpoint
3) The service runs the model and returns a response
4) You parse the response and use it (print, save, analyze)

```
You  →  API endpoint  →  Model runs  →  Response back to you
```

### What “messages” mean here
- We send both a short "system" message (role/context) and a "user" message (your prompt)
- The model uses both to generate the reply


## First-run checklist (you can copy-paste this into your notes)

1) Create an account and an API key (your "library card").
2) Store the key as an environment variable named `OPENAI_API_KEY`.
   - macOS (temporary for current Terminal session):
     - `export OPENAI_API_KEY="sk-..."`
   - macOS (persistent in `~/.zshrc`):
     - Add this line: `export OPENAI_API_KEY="sk-..."` and then run `source ~/.zshrc`
3) Run the connection test cell to verify access.
4) Run the examples; adjust `DEFAULT_MODEL` if desired.
5) If batching many items, estimate tokens first to stay within budget.


## Part 1: Setting Up - Loading API Keys Securely

First, we'll load the necessary libraries and set up secure API key management.

In [ ]:
# Install required packages (run once)
# !pip install openai pandas requests PyPDF2 tqdm python-dotenv

# If you encounter issues, try:
# !pip install openai==0.28.1  # For older stable version
# or
# !pip install --upgrade openai  # For latest version

In [1]:
import os
import getpass
import pandas as pd
import json

# For older OpenAI library (< 1.0)
import openai

# Get API key securely
if os.getenv('OPENAI_API_KEY'):
    openai.api_key = os.getenv('OPENAI_API_KEY')
    print("✅ Using API key from environment")
else:
    api_key = getpass.getpass(prompt="Enter your OpenAI API key: ")
    openai.api_key = api_key
    print("✅ API key set")

print("Ready to make API calls!")

✅ API key set
Ready to make API calls!


## Part 2: Basic API Query - Testing the Connection

We’ll send a simple question and print the model’s reply.

What to notice:
- We pass a short `system` role for context (tone/role of the assistant)
- The `temperature` controls randomness (0 = consistent; 1 = creative)
- You can change `model` to a different one if available

In [2]:
def simple_query(prompt, model="gpt-3.5-turbo", temperature=0.7):
    """
    Send a simple query to the OpenAI API.
    
    Args:
        prompt: The question or prompt to send
        model: The model to use (gpt-3.5-turbo is cost-effective)
        temperature: Controls randomness (0=deterministic, 1=creative)
    
    Returns:
        The model's response as a string
    """
    
    response = openai.ChatCompletion.create(
        model=model,
        messages=[
            {"role": "system", "content": "You are a helpful academic research assistant."},
            {"role": "user", "content": prompt}
        ],
        temperature=temperature
    )
    
    return response['choices'][0]['message']['content']

# Test the connection
test_prompt = "What are the key components of a systematic literature review?"
response = simple_query(test_prompt)
print("Response from AI:")
print(response)

Response from AI:
A systematic literature review typically involves the following key components:

1. Research question or objective: Clearly defining the research question or objective that the review aims to address is essential. This helps guide the search strategy and selection criteria for including studies.

2. Search strategy: Developing a comprehensive search strategy to identify relevant studies is crucial. This involves identifying relevant databases, keywords, and search terms to ensure a thorough and systematic search.

3. Study selection criteria: Establishing specific criteria for selecting studies to be included in the review helps ensure that the review is systematic and focused. These criteria may include factors such as publication date, study design, population characteristics, and outcome measures.

4. Data extraction: Systematically extracting relevant data from the selected studies is a key component of a systematic literature review. This typically involves creat

## Part 3: Single Paper Review - Extracting Key Information

We’ll extract a consistent JSON structure from a single paper.

Why JSON?
- Easy to parse in Python, save to CSV, or load into a database
- Consistent fields make downstream analysis simpler (filter, aggregate, visualize)

Prompt tips:
- Say "Return ONLY valid JSON" to avoid extra text
- Enumerate keys you want back (e.g., `research_question`, `methodology`, `key_findings`)

In [3]:
def analyze_single_paper(paper_title, paper_abstract):
    """
    Analyze a single academic paper and extract key information.
    
    Args:
        paper_title: The title of the paper
        paper_abstract: The abstract of the paper
    
    Returns:
        A dictionary with extracted information
    """
    
    prompt = f"""
    Please analyze this academic paper:
    
    Title: {paper_title}
    
    Abstract: {paper_abstract}
    
    Extract the following in JSON format:
    - research_question: Main research question
    - methodology: Brief description of methods
    - key_findings: List of 3-5 main findings
    - implications: Practical implications
    - limitations: Any limitations mentioned
    
    Return ONLY the JSON object.
    """
    
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are an academic paper analyst. Return only JSON."},
            {"role": "user", "content": prompt}
        ],
        temperature=0  # Use 0 for consistent extraction
    )
    
    result_text = response['choices'][0]['message']['content'].strip()
    
    # Clean up JSON if needed
    if result_text.startswith("```json"):
        result_text = result_text[7:-3]
    
    try:
        return json.loads(result_text)
    except:
        return {"error": "Failed to parse response", "raw": result_text}

# Example usage
sample_title = "The Impact of AI Tools on Academic Research Productivity"
sample_abstract = """
This study examines the impact of AI tools on academic research productivity. 
Through surveys (n=500) with researchers, we find that AI tools reduce time 
spent on literature reviews by 40% and data analysis by 35%. However, concerns 
about accuracy and over-reliance on AI emerge as key challenges.
"""

result = analyze_single_paper(sample_title, sample_abstract)
print("Analysis Result:")
print(json.dumps(result, indent=2))

Analysis Result:
{
  "research_question": "To what extent do AI tools impact academic research productivity?",
  "methodology": "Surveys were conducted with 500 researchers to gather data on the use of AI tools in academic research.",
  "key_findings": [
    "AI tools reduce time spent on literature reviews by 40%.",
    "AI tools reduce time spent on data analysis by 35%.",
    "Concerns about accuracy and over-reliance on AI are key challenges."
  ],
  "implications": "The findings suggest that AI tools can significantly improve research productivity but also raise concerns that need to be addressed.",
  "limitations": "The study relies on self-reported data from researchers and may be subject to response bias."
}


## Part 4: Batch Processing - Analyzing Multiple Papers

We’ll loop over many items and collect results.

What to notice:
- A `for` loop iterates paper-by-paper
- `tqdm` adds a progress bar so students see progress and timing
- `try/except` ensures one failure doesn’t stop the whole batch

Teaching tip:
- Start with 2–3 items to keep it fast, then scale up

In [4]:
from tqdm import tqdm

def process_paper_batch(papers_list):
    """
    Process multiple papers in a loop.
    
    Args:
        papers_list: List of dictionaries with 'title' and 'abstract'
    
    Returns:
        List of analysis results
    """
    
    results = []
    
    # Process each paper with a progress bar
    for paper in tqdm(papers_list, desc="Processing papers"):
        try:
            result = analyze_single_paper(
                paper_title=paper['title'],
                paper_abstract=paper['abstract']
            )
            results.append(result)
        except Exception as e:
            results.append({"error": str(e)})
    
    return results

# Sample papers for demonstration
sample_papers = [
    {
        'title': 'Machine Learning in Healthcare',
        'abstract': 'This review examines ML applications in healthcare diagnosis and treatment.'
    },
    {
        'title': 'Climate Change and Economic Growth',
        'abstract': 'Using panel data from 150 countries, we study climate impacts on GDP.'
    },
    {
        'title': 'Digital Divide in Education',
        'abstract': 'COVID-19 exposed disparities in digital learning access across demographics.'
    }
]

# Process the batch
print("Processing batch of papers...")
results = process_paper_batch(sample_papers)

# Display results
for i, result in enumerate(results):
    print(f"\nPaper {i+1}:")
    print(json.dumps(result, indent=2))

Processing batch of papers...


Processing papers: 100%|██████████| 3/3 [00:05<00:00,  1.78s/it]


Paper 1:
{
  "research_question": "To examine machine learning applications in healthcare diagnosis and treatment",
  "methodology": "Review of existing literature and case studies on machine learning in healthcare",
  "key_findings": [
    "Machine learning can improve diagnostic accuracy in healthcare",
    "Machine learning can assist in personalized treatment plans for patients",
    "Machine learning can help in predicting patient outcomes"
  ],
  "implications": "The findings suggest that integrating machine learning in healthcare can lead to more accurate diagnoses and personalized treatment plans, ultimately improving patient outcomes.",
  "limitations": "One limitation mentioned is the need for large and diverse datasets for training machine learning models in healthcare."
}

Paper 2:
{
  "research_question": "To study climate impacts on GDP",
  "methodology": "Panel data analysis from 150 countries",
  "key_findings": [
    "Climate change has a negative impact on economic g

## Part 5: Advanced Literature Review with Relevance Scoring

We’ll score each paper 1–5 for relevance to a research question and ask for a short explanation.

Rubric idea:
- 1–2: Tangential or off-topic
- 3: Related but not a strong match
- 4: Good match with several direct connections
- 5: Strong, central match to the question

Use cases:
- Triage a large set quickly
- Prioritize what to read first

In [ ]:
# Download and prepare the COVIDiSTRESS dataset
# Dataset available at: https://osf.io/z39us/

# For workshop, we'll simulate similar open-ended responses
# In practice, download the actual CSV from the OSF repository

# Create sample open-ended responses similar to COVIDiSTRESS survey
sample_responses = [
    {
        'id': 1,
        'response': "Working from home has been challenging. I miss my colleagues and the office environment. The isolation is affecting my mental health.",
        'country': 'USA'
    },
    {
        'id': 2,
        'response': "I'm worried about my elderly parents. Can't visit them due to restrictions. Financial stress from reduced work hours.",
        'country': 'UK'
    },
    {
        'id': 3,
        'response': "Lost my job due to lockdown. Struggling to pay rent. But spending more time with family has been positive.",
        'country': 'India'
    },
    {
        'id': 4,
        'response': "Healthcare system is overwhelmed. As a nurse, I'm exhausted and scared. Need more PPE and support.",
        'country': 'Italy'
    },
    {
        'id': 5,
        'response': "Online learning is difficult for my children. Juggling work and homeschooling. Technology issues and lack of social interaction for kids.",
        'country': 'Canada'
    }
]

# Convert to DataFrame
import pandas as pd
responses_df = pd.DataFrame(sample_responses)
print(f"Loaded {len(responses_df)} open-ended responses")
responses_df.head()

## Part 7: Real-World Example - Categorizing Open-Ended Survey Responses

Let's work with real survey data containing open-ended responses. We'll use the COVIDiSTRESS Global Survey dataset which includes written experiences from participants during the COVID-19 pandemic.

### Dataset Information
- **Source**: COVIDiSTRESS Global Survey (N=173,426)
- **Period**: March 30 - May 30, 2020
- **Contains**: Demographics, stress measures, and open-ended written experiences
- **Download**: Available from Nature Scientific Data

In [5]:
def evaluate_paper_relevance(paper, research_question):
    """
    Evaluate how relevant a paper is to your research question.
    
    Args:
        paper: Dictionary with 'title' and 'abstract'
        research_question: Your research question
    
    Returns:
        Dictionary with relevance score and explanation
    """
    
    prompt = f"""
    Research Question: {research_question}
    
    Paper Title: {paper['title']}
    Abstract: {paper['abstract']}
    
    Rate this paper's relevance (1-5) and explain why.
    
    Return JSON with:
    - relevance_score: 1-5 (1=not relevant, 5=highly relevant)
    - explanation: Brief explanation
    - key_connections: List of 2-3 connections to research question
    """
    
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are evaluating academic paper relevance. Return only JSON."},
            {"role": "user", "content": prompt}
        ],
        temperature=0
    )
    
    result = response['choices'][0]['message']['content'].strip()
    if result.startswith("```json"):
        result = result[7:-3]
    
    return json.loads(result)

# Your research question
my_research_question = """
How do AI tools like ChatGPT impact academic research productivity 
in literature reviews and data analysis?
"""

# Evaluate each sample paper
print(f"Research Question: {my_research_question}\n")
print("="*50)

for paper in sample_papers:
    relevance = evaluate_paper_relevance(paper, my_research_question)
    print(f"\nPaper: {paper['title']}")
    print(f"Relevance: {relevance['relevance_score']}/5")
    print(f"Explanation: {relevance['explanation']}")

Research Question: 
How do AI tools like ChatGPT impact academic research productivity 
in literature reviews and data analysis?



Paper: Machine Learning in Healthcare
Relevance: 3/5
Explanation: The paper is somewhat relevant as it discusses machine learning applications, but the focus is on healthcare rather than academic research productivity in literature reviews and data analysis.

Paper: Climate Change and Economic Growth
Relevance: 3/5
Explanation: The paper's focus on climate impacts on GDP may not directly address the impact of AI tools on academic research productivity, but it provides relevant insights into the broader topic of climate change and economic growth.

Paper: Digital Divide in Education
Relevance: 3/5
Explanation: The paper's focus on digital disparities in education is somewhat relevant to the research question about AI tools impacting academic research productivity.


## Part 6: Creating a Literature Review Summary

We’ll create a concise synthesis across papers.

Structure to aim for:
- Common themes across the set
- Key findings (evidence-based)
- Gaps/limitations in current literature
- Future research directions

Tip:
- Keep voice academic and precise; avoid claims not supported by the inputs

In [6]:
def generate_literature_synthesis(papers_analyses, research_question):
    """
    Generate a synthesis of multiple papers for a literature review.
    
    Args:
        papers_analyses: List of analyzed papers
        research_question: Your research question
    
    Returns:
        A structured literature review synthesis
    """
    
    # Format papers for the prompt
    papers_text = "\n\n".join([
        f"Paper {i+1}: {paper.get('title', 'Unknown')}\n"
        f"Findings: {paper.get('key_findings', paper.get('abstract', 'N/A'))}"
        for i, paper in enumerate(papers_analyses)
    ])
    
    prompt = f"""
    Research Question: {research_question}
    
    Papers:
    {papers_text}
    
    Create a literature review synthesis with:
    
    1. COMMON THEMES: What themes appear across papers?
    2. KEY FINDINGS: Main discoveries from these papers
    3. GAPS: What's missing from current research?
    4. FUTURE RESEARCH: What should be studied next?
    
    Keep it concise and academic.
    """
    
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are writing an academic literature review."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.3
    )
    
    return response['choices'][0]['message']['content']

# Generate synthesis using our sample papers and previous analyses
print("Literature Review Synthesis:")
print("="*50)

# Combine our sample papers with any analysis results
papers_for_synthesis = []
for i, paper in enumerate(sample_papers):
    papers_for_synthesis.append({
        'title': paper['title'],
        'abstract': paper['abstract'],
        'key_findings': f"Sample findings for {paper['title']}"
    })

synthesis = generate_literature_synthesis(papers_for_synthesis, my_research_question)
print(synthesis)

Literature Review Synthesis:
Literature Review Synthesis

1. COMMON THEMES:
Across the papers on Machine Learning in Healthcare, Climate Change and Economic Growth, and Digital Divide in Education, common themes emerge regarding the utilization of AI tools like ChatGPT in enhancing academic research productivity. These themes include the integration of AI technologies to streamline literature reviews, improve data analysis processes, and facilitate knowledge synthesis in diverse academic fields.

2. KEY FINDINGS:
- Machine Learning in Healthcare: The application of machine learning algorithms in healthcare has shown promising results in predictive analytics, disease diagnosis, and personalized treatment recommendations.
- Climate Change and Economic Growth: Studies have highlighted the interconnectedness between climate change and economic growth, emphasizing the need for data-driven approaches to address environmental challenges and foster sustainable development.
- Digital Divide in 

## Best Practices and Tips

### 1. API Key Management
- Never hardcode API keys in your code
- Use environment variables or .env files
- Limit who has access; rotate keys if shared

### 2. Cost & Throughput
- Choose a cost-effective default model where possible
- Count tokens for long prompts or big batches
- Batch multiple items in one request when appropriate (vectorized prompt)

### 3. Error Handling
- Wrap API calls in try/except; keep going on failures
- Handle common HTTP codes: 401 (auth), 429 (rate limit), 500 (server)
- Consider short exponential backoff on retryable errors

### 4. Data Quality
- Validate outputs before using them (especially JSON)
- Keep a small labeled set to sanity-check results
- Maintain human oversight for critical judgments

In [ ]:
# Example: Token counting for cost estimation
import tiktoken

def count_tokens(text, model="gpt-4"):
    """Count tokens in a text string for cost estimation."""
    encoding = tiktoken.encoding_for_model(model)
    return len(encoding.encode(text))

# Example text
example_text = "This is a sample text to count tokens."
token_count = count_tokens(example_text)
print(f"Token count: {token_count}")
print(f"Estimated cost (GPT-4): ${token_count * 0.00003:.4f}")  # Adjust pricing as needed

## Workshop Exercises

### Exercise 1: Customize the Analysis
Modify the `analyze_single_paper` function to extract additional information relevant to your field.

### Exercise 2: Build Your Dataset
Create a CSV file with papers from your research area and process them using the batch processing function.

### Exercise 3: Comparative Analysis
Write a function that compares two papers and identifies similarities and differences.

### Exercise 4: Citation Network
Extract citation information and build a network of related papers.

## Prompt templates: edit these in one place

Centralize all prompts here so you can tweak examples without touching the loop or API logic.

- Change wording, tone, and output format in one spot
- Use placeholders like `{paper_title}`, `{paper_abstract}`, `{research_question}`
- Keep output formats rigid when you need machine-readability (e.g., JSON)
- When adding new templates, prefer triple-quoted strings and keep instructions concise

Tip: Print a rendered prompt before sending to the API to sanity-check placeholders and formatting.


In [ ]:
from dataclasses import dataclass
from typing import Dict

@dataclass
class PromptTemplates:
    # Single paper analysis in JSON
    analyze_paper: str = (
        """
        You are an academic paper analyst. Return ONLY valid JSON.
        
        Analyze this paper and extract:
        - research_question
        - methodology
        - key_findings (3-5 bullet points)
        - implications
        - limitations
        
        Paper Title: {paper_title}
        Abstract: {paper_abstract}
        """
    )

    # Relevance scoring with short explanation
    relevance_scoring: str = (
        """
        Return ONLY valid JSON with fields:
        - relevance_score (1-5)
        - explanation (<= 3 sentences)
        - key_connections (2-3 bullets)
        
        Research Question: {research_question}
        Paper Title: {paper_title}
        Abstract: {paper_abstract}
        """
    )

    # Synthesis prompt
    synthesis: str = (
        """
        You are writing a concise academic literature synthesis.
        Given the list of papers, produce:
        1) COMMON THEMES
        2) KEY FINDINGS
        3) GAPS
        4) FUTURE RESEARCH
        
        Research Question: {research_question}
        Papers (title and findings/abstracts):
        {papers_text}
        """
    )

TEMPLATES = PromptTemplates()


## Loop patterns: minimal, step-by-step, and vectorized

These examples separate the loop logic from prompts and API calls. They print intuitive, teaching-friendly outputs:

- What is being iterated
- Which prompt is being used
- What the model returned (pretty-printed)
- How to switch models

What each function demonstrates:
- `loop_minimal`: the smallest working `for` loop + prompt
- `loop_step_by_step`: prints each step, the prompt preview, and the result
- `loop_vectorized_single_prompt`: combines multiple items into a single request for efficiency


## No-API mode: mock model for teaching

Use the mock to teach loops and prompt design without any external calls.
- Toggle `DRY_RUN = True`
- The mock will show a prompt preview and a fake response header
- Swap to real calls later by flipping `DRY_RUN = False`


In [ ]:
# Demo: zero-API exercises

# 1) Minimal loop with mock output
loop_minimal(sample_papers)

# 2) Step-by-step loop with mock output
loop_step_by_step(sample_papers, research_question=my_research_question)

# 3) Vectorized single prompt with mock output
loop_vectorized_single_prompt(sample_papers)


## Glossary (1-minute read)

- **API**: A way for programs to talk to services
- **API key**: Your ID card for the service
- **Prompt**: The instruction or question you send to the model
- **Tokens**: Chunks of text used for billing/limits
- **Temperature**: Controls randomness (lower = more consistent)
- **Rate limit**: The maximum requests allowed in a time window (handle 429 errors with short delays)


## Troubleshooting and privacy notes

- 401 Unauthorized: check `OPENAI_API_KEY` is set correctly
- 429 Rate limit: wait briefly and retry; reduce request rate or batch items
- 500 Server error: retry after a short delay; if persistent, try later
- Validate outputs: treat AI responses as drafts to review, not truth
- Avoid sending sensitive data; anonymize where possible
- For JSON outputs, keep prompts strict and add fallback parsing if needed
